In [7]:
import pandas as pd
import os

# Path to the directory containing chunk files
folder_path = r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\Chunks"

# List all CSV files in the folder
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# Read and concatenate all CSVs into one DataFrame
transactions_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# Preview the result
print(transactions_df.head())


   step       customer  age gender zipcodeOri       merchant zipMerchant  \
0     0  'C1093826151'  '4'    'M'    '28007'   'M348934600'     '28007'   
1     0   'C352968107'  '2'    'M'    '28007'   'M348934600'     '28007'   
2     0  'C2054744914'  '4'    'F'    '28007'  'M1823072687'     '28007'   
3     0  'C1760612790'  '3'    'M'    '28007'   'M348934600'     '28007'   
4     0   'C757503768'  '5'    'M'    '28007'   'M348934600'     '28007'   

              category  amount  fraud  
0  'es_transportation'    4.55      0  
1  'es_transportation'   39.68      0  
2  'es_transportation'   26.89      0  
3  'es_transportation'   17.25      0  
4  'es_transportation'   35.72      0  


In [8]:
import pandas as pd
from datetime import datetime
import os

# transactions_df = pd.read_csv(r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\Chunks\chunk_1_20250624_202535.csv")
customer_df = pd.read_csv(r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\customer data\customer data.csv")   

In [9]:
merged_df = transactions_df.merge(
    customer_df,
    how='inner',
    left_on=['customer', 'merchant', 'category','amount'],
    right_on=['Source', 'Target', 'typeTrans','Weight']
)
merged_df

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud_x,Source,Target,Weight,typeTrans,fraud_y
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0,'C1093826151','M348934600',4.55,'es_transportation',0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0,'C352968107','M348934600',39.68,'es_transportation',0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0,'C2054744914','M1823072687',26.89,'es_transportation',0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0,'C1760612790','M348934600',17.25,'es_transportation',0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0,'C757503768','M348934600',35.72,'es_transportation',0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91062,33,'C608244416','1','F','28007','M348934600','28007','es_transportation',37.18,0,'C608244416','M348934600',37.18,'es_transportation',0
91063,33,'C608244416','1','F','28007','M1823072687','28007','es_transportation',25.87,0,'C608244416','M1823072687',25.87,'es_transportation',0
91064,33,'C1031698099','0','M','28007','M1823072687','28007','es_transportation',49.90,0,'C1031698099','M1823072687',49.90,'es_transportation',0
91065,33,'C602544155','4','M','28007','M348934600','28007','es_transportation',17.03,0,'C602544155','M348934600',17.03,'es_transportation',0


In [10]:
from datetime import datetime

def detect_pattern3(transactions_df):
    detections = []

    # Step 1: Drop duplicate customer-merchant-gender to get unique customers per gender per merchant
    deduped = transactions_df[['merchant', 'customer', 'gender']].drop_duplicates()

    # Step 2: Group by merchant and gender
    gender_counts = deduped.groupby(['merchant', 'gender'])['customer'].nunique().unstack(fill_value=0).reset_index()

    # Step 3: Rename columns for safety
    gender_counts.columns.name = None
    gender_counts = gender_counts.rename(columns={'F': 'female_count', 'M': 'male_count'})

    # Fill missing columns if needed (e.g. if only 'M' or only 'F' exists for a merchant)
    if 'female_count' not in gender_counts:
        gender_counts['female_count'] = 0
    if 'male_count' not in gender_counts:
        gender_counts['male_count'] = 0

    # Step 4: Apply filter condition
    filtered = gender_counts[
        (gender_counts['female_count'] > 100) &
        (gender_counts['female_count'] < gender_counts['male_count'])
    ]

    # Step 5: Construct detections
    detection_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    y_start_time = detection_time  # assuming same for now

    for _, row in filtered.iterrows():
        detection = {
            "YStartTime": y_start_time,
            "detectionTime": detection_time,
            "patternId": "PatId3",
            "ActionType": "DEI-NEEDED",
            "customerName": "",
            "MerchantId": row['merchant']
        }
        detections.append(detection)

    return pd.DataFrame(detections)


In [11]:
pattern3_detections_df = detect_pattern3(transactions_df)
print(pattern3_detections_df.head())


Empty DataFrame
Columns: []
Index: []


In [12]:
pattern3_detections_df

""
